In [19]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
import numpy as np
import pymysql

In [20]:
#conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='101Monkeybob!',db='Final_Project')
#df=pd.read_sql_query("SELECT * FROM 'house' ",conn)


In [21]:
# Import Clean House Data
house_data = pd.read_csv("clean_house_data_ML.csv",low_memory = False)
imp_df = pd.DataFrame(house_data)
imp_df.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,PRICE,INDEX_BIN
0,3,1,8,3,2010.0,1.0,0,1424,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,1325000.0,52
1,1,0,6,3,2006.0,1.0,0,2090,20009.0,2,7.875,0.0,1.0,0.0,1.0,0.0,0.0,33232.0,1
2,2,1,5,3,2012.0,1.0,1,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,907400.0,36
3,2,1,8,3,2012.0,1.0,2,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,1065000.0,42
4,2,1,7,2,2009.0,1.0,1,838,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,216000.0,8


In [22]:
# Dropping rows with a saledate less than 2000
new_df = imp_df[imp_df.SALEDATE >= 2000]
new_df.head(50)

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,PRICE,INDEX_BIN
0,3,1,8,3,2010.0,1.0,0,1424,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,1325000.0,52
1,1,0,6,3,2006.0,1.0,0,2090,20009.0,2,7.875000,0.0,1.0,0.0,1.0,0.0,0.0,33232.0,1
2,2,1,5,3,2012.0,1.0,1,2090,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,907400.0,36
3,2,1,8,3,2012.0,1.0,2,2090,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,1065000.0,42
4,2,1,7,2,2009.0,1.0,1,838,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,216000.0,8
5,2,1,6,2,2005.0,1.0,0,1700,20009.0,2,6.375000,0.0,1.0,0.0,1.0,0.0,0.0,990000.0,39
6,2,1,7,3,2014.0,1.0,2,1000,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,682500.0,27
7,1,1,6,3,2005.0,1.0,0,1424,20009.0,2,6.375000,0.0,1.0,0.0,1.0,0.0,0.0,993500.0,39
8,2,1,6,3,2012.0,1.0,0,1424,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,745000.0,29
9,3,1,9,3,2006.0,1.0,1,1424,20009.0,2,7.875000,0.0,1.0,0.0,1.0,0.0,0.0,277000.0,11


In [23]:
new_df = new_df.drop(columns=['PRICE'])
new_df

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,INDEX_BIN
0,3,1,8,3,2010.0,1.0,0,1424,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,52
1,1,0,6,3,2006.0,1.0,0,2090,20009.0,2,7.875000,0.0,1.0,0.0,1.0,0.0,0.0,1
2,2,1,5,3,2012.0,1.0,1,2090,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,36
3,2,1,8,3,2012.0,1.0,2,2090,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,42
4,2,1,7,2,2009.0,1.0,1,838,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37013,2,1,7,3,2008.0,1.0,1,1734,20032.0,8,4.928571,0.0,1.0,0.0,0.0,1.0,0.0,9
37014,1,1,9,3,2001.0,1.0,0,2799,20032.0,8,6.772727,0.0,1.0,0.0,0.0,1.0,0.0,8
37015,1,1,7,3,2013.0,1.0,0,2359,20032.0,8,3.250000,0.0,1.0,0.0,0.0,1.0,0.0,2
37016,1,1,7,3,2012.0,1.0,0,1919,20032.0,8,3.250000,0.0,1.0,0.0,0.0,1.0,0.0,5


In [24]:
# Define the features 
X = new_df.copy()
X = X.drop("INDEX_BIN", axis = 1)
X.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
0,3,1,8,3,2010.0,1.0,0,1424,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
1,1,0,6,3,2006.0,1.0,0,2090,20009.0,2,7.875,0.0,1.0,0.0,1.0,0.0,0.0
2,2,1,5,3,2012.0,1.0,1,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
3,2,1,8,3,2012.0,1.0,2,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
4,2,1,7,2,2009.0,1.0,1,838,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0


In [25]:
# Define target set
y = new_df['INDEX_BIN'].ravel()
y[:5]

array([52,  1, 36, 42,  8], dtype=int64)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [27]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators = 100, random_state = 2)
model.fit(X_train,y_train)

BalancedRandomForestClassifier(random_state=2)

In [28]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
accuracy_score = balanced_accuracy_score(y_test, y_pred)
accuracy_score

0.02886362628753867

In [29]:
#Fitting the model
confusion_matrix(y_test,y_pred)

array([[ 7, 12,  9, ...,  0,  2,  0],
       [16, 22,  5, ...,  1,  0,  1],
       [24, 23, 14, ...,  5,  0,  3],
       ...,
       [ 0,  0,  0, ...,  1,  1,  1],
       [ 0,  0,  0, ...,  2,  1,  2],
       [ 0,  0,  0, ...,  0,  3,  0]], dtype=int64)

In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.01      0.10      0.94      0.02      0.31      0.09        70
          2       0.05      0.17      0.96      0.08      0.41      0.15       126
          3       0.05      0.06      0.97      0.06      0.25      0.06       224
          4       0.06      0.04      0.97      0.05      0.21      0.04       323
          5       0.02      0.01      0.98      0.02      0.11      0.01       333
          6       0.03      0.03      0.98      0.03      0.16      0.02       264
          7       0.07      0.05      0.98      0.06      0.22      0.04       258
          8       0.07      0.03      0.99      0.04      0.18      0.03       265
          9       0.05      0.02      0.99      0.03      0.14      0.02       309
         10       0.05      0.03      0.98      0.04      0.18      0.03       234
         11       0.02      0.01      0.99      0.01      0.10      0.01       284
   

In [31]:
# List the features sorted in descending order by feature importance
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.23223286445990474, 'LANDAREA'),
 (0.14170152299007158, 'SALEDATE'),
 (0.10630551334777108, 'ROOMS'),
 (0.10091925750144655, 'PRIME'),
 (0.07446397881198048, 'ZIPCODE'),
 (0.06561700790445116, 'BEDRM'),
 (0.06537503441294898, 'BATHRM'),
 (0.06144107515033922, 'HF_BATHRM'),
 (0.0492910654520084, 'WARD'),
 (0.042654295928747255, 'FIREPLACES'),
 (0.020211092069730652, 'AC_Y'),
 (0.01982098086480394, 'AC_N'),
 (0.007134240751044721, 'QUADRANT_NE'),
 (0.006726246665001115, 'QUADRANT_SE'),
 (0.004413201224527298, 'QUADRANT_NW'),
 (0.0016926224652227587, 'QUADRANT_SW'),
 (0.0, 'KITCHENS')]